# Transitions and Networks

Different path sampling approaches use different kinds of ensembles. But what they all share in common is that those ensembles represent some network of transitions.

Under most circumstances, the user can directly work with `TransitionNetwork`s, and never needs to work directly with `Ensemble`s. This document describes the internal structure of `TransitionsNetwork`s, and explains how to customize that structure with different ensembles if so desired.

This also covers some of the interface between `TransitionNetwork`s and `MoveStrategy` objects. When developing new methodologies, this interface allows for high levels of flexibility. A new `TransitionNetwork` could be combined with a customized `MoveStrategy` to develop all sorts of path-sampling-like approaches.

## `Network`s are made of `Transition`s

A transition describes a set of paths with certain allowed initial and final conditions. There are two groups of transitions that are important when working with networks. First, there are the `sampling_transitions`, which are what are actually sampled by the dynamics. Then there are `analysis_transitions` (or just `transitions`) which are the subjects of analysis.

The difference between these is easily described in the context of MSTIS. With MSTIS, you use one set of ensembles to sample the transitions from a given state to all other states. So you would sample $A\to (B \text{ or } C)$, but the analysis transitions would include separate studies of $A\to B$ and $A\to C$.

The network is defined by the combination of what the underlying transitions to study are, and the approach used to study them. It defines a set of path ensembles which are sampled during the dynamics, and the way to combine those afterward into results that connect to physical meaning.

### Handcrafting `Network`s

Beyond the simple tools to customize the networks you study, you can manually create them with whatever complexity you desire. The `Network` level of code is designed to interact with the `MoveScheme` and `MoveStrategy` code, as well as with some of the analysis tools. As long as you implement a few items, your custom `Network` will work seamlessly with those.

The main point is that there are two aspects of the network: there's the sampling network, which is organizes the ensembles

#### Handcrafting the sampling network

The sampling network consists of three objects:

* `_sampling_transitions`: a list of transitions; contains all the normal ensembles
* `special_ensembles`: a dictionary with strings for keys, describing the type of ensemble, and dictionaries for values. Those value dictionaries have the ensembles themselves as keys, and a list of associated transitions as values.
* `hidden_ensembles`: this list is empty when built by the network, and is set by the MoveScheme if necessary

The special ensembles need to have the correct key names to work with the `MoveScheme` and `MoveStrategy` subsystem. These are `'minus'` for the minus interfaces, and `'ms_outer'` for the multiple state outer interfaces.

If you do that, then `MoveStrategy` and `MoveScheme` will work with your networks.

#### Handcrafting the analysis network

